In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from transformers import pipeline, BertTokenizer
from tqdm import tqdm
import nltk
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import re, string
import os.path

#! pip install nltk==3.3

df = pd.read_json('dataframes/reddit_data.json')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 14770 entries, c1e4o to c0i10ti
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   score             14770 non-null  int64         
 1   controversiality  14770 non-null  int64         
 2   subreddit         14770 non-null  object        
 3   body              14770 non-null  object        
 4   month             14770 non-null  int64         
 5   year              14770 non-null  int64         
 6   original_size     14770 non-null  int64         
 7   PS                14770 non-null  int64         
 8   XBOX              14770 non-null  int64         
 9   PS_Count          14770 non-null  float64       
 10  XBOX_Count        14770 non-null  float64       
 11  date              14770 non-null  datetime64[ns]
 12  naive_sentiment   14770 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(7), object(2)
memory usage: 1.6+ MB

To start off, we'll be needing a value to predict.  I'm going to be using a secondary sentiment analysis through a bert classifier that ranks the sentiment on a scale of 1-5 stars, as well as gives a confidence score.  Since this is a large file, I've broken up the data into parts, saved them, and recombined them after adding the new columns.

In [2]:
classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')
max_length = 512

years = ['06', '07', '08', '09']

for year in years:
    if os.path.exists('dataframes/df'+year+'_bert.json'):
        print('dataframes/df'+year+'_bert.json exists')
    else:
        labels = []
        scores = []
        mini_df = df[df['year']==2000+int(year)]
        for post in tqdm(mini_df['body']):
            a = classifier(post, max_length=max_length, truncation=True) #this returns a 1-element list of a dictionary
            labels.append(a[0]['label'][0]) #a[0]['label'] returns a label between '1 star' and '5 stars', I only need the integer
            scores.append(a[0]['score']) #confidence score of said label
        mini_df['bert_labels'] = labels
        mini_df['bert_scores'] = scores
        mini_df.to_json('dataframes/df'+year+'_bert.json')

dataframes/df06_bert.json exists
dataframes/df07_bert.json exists
dataframes/df08_bert.json exists
dataframes/df09_bert.json exists


In [3]:
#recombining data after the split.
df06 = pd.read_json('dataframes/df06_bert.json')
df07 = pd.read_json('dataframes/df07_bert.json')
df08 = pd.read_json('dataframes/df08_bert.json')
df09 = pd.read_json('dataframes/df09_bert.json')

df = df06.append(df07.append(df08.append(df09)))
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 14770 entries, c1e4o to c0i10ti
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   score             14770 non-null  int64         
 1   controversiality  14770 non-null  int64         
 2   subreddit         14770 non-null  object        
 3   body              14770 non-null  object        
 4   month             14770 non-null  int64         
 5   year              14770 non-null  int64         
 6   original_size     14770 non-null  int64         
 7   PS                14770 non-null  int64         
 8   XBOX              14770 non-null  int64         
 9   PS_Count          14770 non-null  float64       
 10  XBOX_Count        14770 non-null  float64       
 11  date              14770 non-null  datetime64[ns]
 12  naive_sentiment   14770 non-null  float64       
 13  bert_labels       14770 non-null  int64         
 14  bert_scores       147

Now that that's out of the way I need to decide which columns will be useful for my model.

score should have an impact, though controversiality might not since it's effectively a boolean value with a large majority of values being equal to 0.
body will definitely make an impact, we'll need to make some serious changes to make it numeric.
month, year, and date are mostly for the time series I made previously, and will not add much.
original_size, PS_Count, and XBOX_Count were columns I added for plotting, and have no purpose here.
naive_sentiment is definitely important.
PS, XBOX, and subreddit are categoricals that should aid in predictions.
bert_labels are going to be the prediction value and bert_scores will supplement that.

In [4]:
df.drop(columns=['PS_Count', 'controversiality', 'XBOX_Count', 'original_size', 'month', 'year', 'date'], inplace=True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 14770 entries, c1e4o to c0i10ti
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   score            14770 non-null  int64  
 1   subreddit        14770 non-null  object 
 2   body             14770 non-null  object 
 3   PS               14770 non-null  int64  
 4   XBOX             14770 non-null  int64  
 5   naive_sentiment  14770 non-null  float64
 6   bert_labels      14770 non-null  int64  
 7   bert_scores      14770 non-null  float64
dtypes: float64(2), int64(4), object(2)
memory usage: 1.0+ MB
None


Next, we need to address the score column's outliers.  Half of all values are between 0 and 3, but the overall range is from -45 to 398.  The graph from the EDA shows that the variance has been increasing over time and has a non-normal distribution.  If we only keep rows between -30 and 30, we only lose 161 columns in the process, and in doing so make score a more reliable predictor.

In [5]:
df.drop(df[(df['score'] > 30) | (df['score'] < -30)].index, inplace=True)
print(df.describe())

              score            PS          XBOX  naive_sentiment  \
count  14609.000000  14609.000000  14609.000000     14609.000000   
mean       2.132453      0.448080      0.610446         0.077496   
std        4.001580      0.497314      0.487666         0.221732   
min      -30.000000      0.000000      0.000000        -1.000000   
25%        1.000000      0.000000      0.000000        -0.006667   
50%        1.000000      0.000000      1.000000         0.056481   
75%        3.000000      1.000000      1.000000         0.194924   
max       30.000000      1.000000      1.000000         1.000000   

        bert_labels   bert_scores  
count  14609.000000  14609.000000  
mean       2.486412      0.436582  
std        1.465876      0.145641  
min        1.000000      0.208104  
25%        1.000000      0.329384  
50%        2.000000      0.401645  
75%        4.000000      0.508862  
max        5.000000      0.980305  


Excellent, this dropped the std of score from 11.1 to 4.001, while maintaining the percentiles and only changing the mean by about 0.8.  The last thing we need to address is the body and subreddit columns.  We should be able to create dummies for subreddit just fine, but the sheer number of unique words in body would add around 19,000 columns to the database as we'll see later.

In [6]:
df = pd.get_dummies(data=df, columns=['subreddit']) #consider substituting using value count before dummies
small_col = []
total_col = 0
for col in df.columns:
    num_posts = df[col][df[col]==True].count()
    print(col, num_posts)
    total_col += 1
    if num_posts <= 50: #potentially pick bigger number
        small_col.append(col)

score 5842
body 0
PS 6546
XBOX 8918
naive_sentiment 51
bert_labels 5671
bert_scores 0
subreddit_4chan 1
subreddit_AmericanGovernment 1
subreddit_AmericanPolitics 1
subreddit_Anarchism 5
subreddit_Android 41
subreddit_Art 1
subreddit_AskReddit 1250
subreddit_Astronomy 1
subreddit_BDSMcommunity 1
subreddit_Baking 1
subreddit_Borderlands 6
subreddit_Christianity 2
subreddit_CommonLaw 2
subreddit_DAE 2
subreddit_DIY 2
subreddit_Design 1
subreddit_DoesAnybodyElse 41
subreddit_Drugs 1
subreddit_Economics 27
subreddit_Equality 4
subreddit_Eve 2
subreddit_FashionTechnology 1
subreddit_Favors 4
subreddit_FreeMicrosoftPoints 1
subreddit_Frugal 20
subreddit_GameDeals 3
subreddit_Games 1
subreddit_HappyBirthday 1
subreddit_Health 1
subreddit_Homebrewing 1
subreddit_IAmA 224
subreddit_ILiveIn 1
subreddit_IndieGaming 4
subreddit_Israel 1
subreddit_JRPG 4
subreddit_Libertarian 11
subreddit_MW2 11
subreddit_MapleLinks 8
subreddit_Marijuana 74
subreddit_MensRights 5
subreddit_Music 12
subreddit_NSFW_no

Creating dummies for subreddit alone brought us from 6 columns up to 222 columns, most of which have 50 mentions or less.  To avoid overfitting, I will be merging these columns.

In [7]:
#leave out the non-subreddit columns so we don't delete them
small_col.remove('body')
small_col.remove('bert_scores')

#row-wise aggregation of columns within small_col
df['subreddit_other'] = df[small_col].aggregate('sum', axis=1)
df = df.drop(columns=small_col)
print(df.columns)

Index(['score', 'body', 'PS', 'XBOX', 'naive_sentiment', 'bert_labels',
       'bert_scores', 'subreddit_AskReddit', 'subreddit_IAmA',
       'subreddit_Marijuana', 'subreddit_PS3', 'subreddit_WTF',
       'subreddit_business', 'subreddit_entertainment', 'subreddit_funny',
       'subreddit_gadgets', 'subreddit_gaming', 'subreddit_geek',
       'subreddit_linux', 'subreddit_pics', 'subreddit_politics',
       'subreddit_programming', 'subreddit_reddit.com', 'subreddit_science',
       'subreddit_technology', 'subreddit_worldnews', 'subreddit_xbox360',
       'subreddit_other'],
      dtype='object')


28 columns is a lot more manageable, especially since we'll be adding more soon.  Onto working with the body column.  First we need to reduce the body section into tokens, rather than paragraphs.

In [8]:
#tokenizing
superstring = ''

for entry in df['body']:
    superstring += entry

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
vocabulary = tokenizer.tokenize(superstring)

print(vocabulary[0:50])

['the', 'article', 'raises', 'concerns', 'that', 'a', 'single', 'speed', 'blur', '##ay', 'drive', 'will', 'be', 'too', 'slow', 'but', 'no', 'one', 'has', 'even', 'announced', 'a', 'single', 'speed', 'blur', '##ay', 'drive', 'most', 'of', 'them', 'are', '4', '##x', 'so', 'far', 'since', 'there', 'aren', '##t', 'even', 'sing', '##el', 'speed', 'drives', 'available', 'it', 'seems', 'unlikely', 'sony', 'would']


Next we need to remove stopwords and reduce words to their base parts.

In [9]:
#lemmatizing and removing stopwords
def lemma(tokens):
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words('english')
    line = []
    for word, tag in pos_tag(tokens):
        #word = re.sub(regex,'', word) #if I want to get rid of words within a certain regex, like urls
        
        if tag.startswith('NN'): #labels words as a noun
            pos = 'n'
        elif tag.startswith('VB'): #labels words as a verb
            pos = 'v'
        else:
            pos = 'a'
        
        if len(word) > 0 and word not in string.punctuation and word.lower() not in stop_words:
            line.append(lemmatizer.lemmatize(word, pos))
    return(line)

In [10]:
tokens = lemma(vocabulary)
token_series = pd.Series(tokens)
token_series = token_series.value_counts()
print(tokens[0:50])
print(token_series.head())

['article', 'raise', 'concern', 'single', 'speed', 'blur', '##ay', 'drive', 'slow', 'one', 'even', 'announce', 'single', 'speed', 'blur', '##ay', 'drive', '4', '##x', 'far', 'since', '##t', 'even', 'sing', '##el', 'speed', 'drive', 'available', 'seem', 'unlikely', 'sony', 'would', 'use', 'come', 'cost', 'large', 'part', 'investment', 'already', 'make', 'fa', '##bs', 'develop', 'cell', 'processor', 'together', 'ibm', '##shi', '##ba', 'sony']
xbox    11980
game    11322
##t     10410
##s     10274
p        8182
dtype: int64


To decide on which words we will make columns out of, I'm going to use a Term Frequency / Inverse Document Frequency model to score words based off of how frequent they are in both individual posts and the database as a whole.  We'll need the IDF for each token first.  The IDF is basically a measure of how rare a word is in the database, and as a result words with high IDF's have a stronger influence on the features made from each paragraph.

In [11]:
#calculate IDF
for i in df.index: #need to reduce body column into a string of its tokens
    entry = df.loc[i]['body']
    body_tokens = tokenizer.tokenize(entry)
    body_lemma = lemma(body_tokens)
    restring = ''
    for token in body_lemma:
        restring += token + ' '
    df.loc[i, 'body'] = restring

In [12]:
IDF_list = []
total_posts = df['body'].count()

for token in tqdm(token_series.index):
    total_posts_with_token = df['body'][df['body'].str.contains(token)].count()
    if total_posts_with_token > 1:
        IDF = np.log(total_posts / total_posts_with_token)
        IDF_list.append([token, IDF])
        
TF_IDF_df = pd.DataFrame(IDF_list)

100%|███████████████████████████████████████████████████████████████████████████| 15780/15780 [02:32<00:00, 103.43it/s]


In [13]:
TF_IDF_df.rename(columns={0:'token', 1:'IDF'}, inplace=True)
print(TF_IDF_df.sort_values(by='IDF', ascending=False).head(20))

             token       IDF
10619    inspector  8.896246
10989   contractor  8.896246
10977       ##iger  8.896246
10978    endurance  8.896246
10979      postage  8.896246
10980       ##rave  8.896246
10981        paula  8.896246
10982  prehistoric  8.896246
10983       ##sier  8.896246
10984        ##ans  8.896246
10985   algorithms  8.896246
10986        scots  8.896246
10987        ##wen  8.896246
4259       britney  8.896246
10990         lots  8.896246
10974       ##mple  8.896246
10991       ##udes  8.896246
10992      malcolm  8.896246
10993  psychiatric  8.896246
10994      eclipse  8.896246


Now that we have the IDF for each token calculated, time to work on the TF for each post.  The TF measures how frequently each token occurs within one post, and is multiplied with the IDF to generate the importance of a token.

In [14]:
def TF_calc (TF_df, post, total_posts):
    TF_IDF_list = []
    
    for i in TF_df.index:
        token = TF_df.loc[i]['token']
        IDF = TF_df.loc[i]['IDF']
        token_count_in_post = post.count(token)
        
        if token_count_in_post > 0:
            TF = total_posts / token_count_in_post
        else:
            TF = 0
        TF_IDF_list.append([TF * IDF])
        
    return pd.DataFrame(TF_IDF_list, columns=['TFIDF calc'])

An important thing to note is that mapping the full TF-IDF for my database requires 9k x 16k cells, or about 144 million.  To make this easier on my computer, I'm instead only saving the total score for each body and the average TF-IDF of each token, the former will be made into a new column for the main dataframe and the latter will be used to determine which tokens should be used as prediction variables.  Breaking up the loops so I can save progress as I go, since this takes a few hours to run.

In [15]:
loop = [0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000]

#do 1000 at a time, saving as I go for time
for k in loop:
    if os.path.exists('dataframes/TFIDF_'+str(k)+'.json'):
        print('dataframes/TFIDF_'+str(k)+'.json exists')
    else:
        TF_IDF_df['TF_ave'] = 0
        body_tfidf_score = []
        if k == 14000:
            x=df['body'].count() - k
        else:
            x=1000
        
        for i in tqdm(range(x)):
            TF_to_add = TF_calc(TF_IDF_df, df['body'].iloc[i+k], total_posts)
            TF_IDF_df['TF_ave'] += (TF_to_add['TFIDF calc'] / df['body'].count())
            body_tfidf_score.append(TF_to_add['TFIDF calc'].sum())
        TF_IDF_df = TF_IDF_df[TF_IDF_df['TF_ave'] > 0]
        TF_IDF_df.to_json('dataframes/TFIDF_'+str(k)+'.json')
        pd.DataFrame(body_tfidf_score).to_json('dataframes/body_TFIDF_'+str(k)+'.json')

dataframes/TFIDF_0.json exists
dataframes/TFIDF_1000.json exists
dataframes/TFIDF_2000.json exists
dataframes/TFIDF_3000.json exists
dataframes/TFIDF_4000.json exists
dataframes/TFIDF_5000.json exists
dataframes/TFIDF_6000.json exists
dataframes/TFIDF_7000.json exists
dataframes/TFIDF_8000.json exists
dataframes/TFIDF_9000.json exists
dataframes/TFIDF_10000.json exists
dataframes/TFIDF_11000.json exists
dataframes/TFIDF_12000.json exists
dataframes/TFIDF_13000.json exists
dataframes/TFIDF_14000.json exists


In [16]:
ave0 = pd.read_json('dataframes/TFIDF_0.json')
ave1 = pd.read_json('dataframes/TFIDF_1000.json')
ave2 = pd.read_json('dataframes/TFIDF_2000.json')
ave3 = pd.read_json('dataframes/TFIDF_3000.json')
ave4 = pd.read_json('dataframes/TFIDF_4000.json')
ave5 = pd.read_json('dataframes/TFIDF_5000.json')
ave6 = pd.read_json('dataframes/TFIDF_6000.json')
ave7 = pd.read_json('dataframes/TFIDF_7000.json')
ave8 = pd.read_json('dataframes/TFIDF_8000.json')
ave9 = pd.read_json('dataframes/TFIDF_9000.json')
ave10 = pd.read_json('dataframes/TFIDF_10000.json')
ave11 = pd.read_json('dataframes/TFIDF_11000.json')
ave12 = pd.read_json('dataframes/TFIDF_12000.json')
ave13 = pd.read_json('dataframes/TFIDF_13000.json')
ave14 = pd.read_json('dataframes/TFIDF_14000.json')

TF_IDF_df = ave0.append(ave1.append(ave2.append(ave3.append(ave4.append(ave5.append(ave6.append(ave7.append(ave8.append(ave9.append(ave10.append(ave11.append(ave12.append(ave13.append(ave14))))))))))))))

body0 = pd.read_json('dataframes/body_TFIDF_0.json')
body1 = pd.read_json('dataframes/body_TFIDF_1000.json')
body2 = pd.read_json('dataframes/body_TFIDF_2000.json')
body3 = pd.read_json('dataframes/body_TFIDF_3000.json')
body4 = pd.read_json('dataframes/body_TFIDF_4000.json')
body5 = pd.read_json('dataframes/body_TFIDF_5000.json')
body6 = pd.read_json('dataframes/body_TFIDF_6000.json')
body7 = pd.read_json('dataframes/body_TFIDF_7000.json')
body8 = pd.read_json('dataframes/body_TFIDF_8000.json')
body9 = pd.read_json('dataframes/body_TFIDF_9000.json')
body10 = pd.read_json('dataframes/body_TFIDF_10000.json')
body11 = pd.read_json('dataframes/body_TFIDF_11000.json')
body12 = pd.read_json('dataframes/body_TFIDF_12000.json')
body13 = pd.read_json('dataframes/body_TFIDF_13000.json')
body14 = pd.read_json('dataframes/body_TFIDF_14000.json')

body_scores = body0.append(body1.append(body2.append(body3.append(body4.append(body5.append(body6.append(body7.append(body8.append(body9.append(body10.append(body11.append(body12.append(body13.append(body14))))))))))))))

TF_IDF_df = TF_IDF_df.reset_index(drop=True)
body_scores = body_scores.reset_index(drop=True)

Time to add the body_score column to the main df and make token count columns for the tokens with the highest average 50 TF-IDF scores.

In [19]:
#print out top tokens
important_tokens = TF_IDF_df[TF_IDF_df['TF_ave'] > 0].sort_values(by='TF_ave', ascending=False).head(100)
print(important_tokens['token'])

18506       360
19988    budget
18827        60
27650      live
4063       stat
          ...  
22075        im
1892         ai
28720      2006
6960          4
9127         jo
Name: token, Length: 100, dtype: object


In [20]:
#make a column out of the body scores
df['body_score'] = 0

for i in range(df['body'].count()):
    df['body_score'].iloc[i] = body_scores[0].iloc[i]

#make columns out of the most important tokens
for token in important_tokens['token']:
    token_list = []
    df[token] = 0
    
    for j in df.index:
        token_list.append(df.loc[j]['body'].count(token))
        
    token_column = pd.DataFrame(token_list)
    
    k = 0
    for i in df.index: #doing it this way because of setting-on-a-copy issues with iloc on df
        df.loc[i, token] = token_column.iloc[k][0]
        k += 1

print(df.head())

       score                                               body  PS  XBOX  \
c1e4o      7  article raise concern single speed blur ##ay d...   0     1   
c3kh6      2  link ##jack http www mac ##world com 2005 09 f...   1     0   
c5iqp      7  feel discover obligation buy playstation ##s i...   1     0   
c5vr3      3  gt make nintendo wii special different xbox 36...   1     1   
c5zhh     -9  ##t know fa ##gs iv ##e never excited gaming i...   0     1   

       naive_sentiment  bert_labels  bert_scores  subreddit_AskReddit  \
c1e4o        -0.055082            2     0.353368                    0   
c3kh6         0.000000            1     0.289909                    0   
c5iqp         0.235417            2     0.332737                    0   
c5vr3         0.110119            3     0.320127                    0   
c5zhh         0.418750            3     0.303217                    0   

       subreddit_IAmA  subreddit_Marijuana  ...  police  op  ap  photo  ct  \
c1e4o               

Now that our data is properly preprocessed, it's time to sending it off to be modeled in the next notebook.

In [21]:
df = df.drop(columns=['body', 'naive_sentiment', 'bert_scores'])
df.to_json('dataframes/preprocessed_df.json')